In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.model_selection import train_test_split
import pickle

In [56]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import os
import datetime
import tensorflow_hub as hub
from tensorflow import keras

In [62]:
input_data = 'Encampment around and under the El Cajon BLVD and 805 freeway overpass. Trash being left in cul de sac on Bancroft Street and several tents present by the 805 Northbound Freeway entrance.'
classification = 'class'

In [63]:
string_array = np.array([input_data])
print('shape', string_array.shape)
print('type', type(string_array))

shape (1,)
type <class 'numpy.ndarray'>


In [64]:
test_array = np.array([classification])
print('shape', test_array.shape)
print('type', type(test_array))

shape (1,)
type <class 'numpy.ndarray'>


In [65]:
input_dataset = tf.data.Dataset.from_tensor_slices((string_array, test_array))

In [66]:
for text, target in input_dataset.take(1):
    print('description: {}, Target: {}'.format(text, target))

description: b'Encampment around and under the El Cajon BLVD and 805 freeway overpass. Trash being left in cul de sac on Bancroft Street and several tents present by the 805 Northbound Freeway entrance.', Target: b'class'


In [67]:
with open('../data/detailed_types.pkl', 'rb') as file:
    detailed_types = pickle.load(file)
    
with open('../data/vals.pkl', 'rb') as file:
    vals = pickle.load(file)

In [68]:
def fetch(text, labels):
        return text, tf.one_hot(target(labels),len(detailed_types))

In [69]:
table = tf.lookup.StaticHashTable(
    initializer=tf.lookup.KeyValueTensorInitializer(
        keys=tf.constant(detailed_types),
        values=tf.constant(vals),
    ),
        default_value=tf.constant(-1),
        name="target_encoding"
)

In [70]:
@tf.function
def target(x):
    return table.lookup(x)

In [71]:
test_data, test_labels = next(iter(input_dataset.map(fetch).batch(1)))

In [72]:
test_data

<tf.Tensor: shape=(1,), dtype=string, numpy=
array([b'Encampment around and under the El Cajon BLVD and 805 freeway overpass. Trash being left in cul de sac on Bancroft Street and several tents present by the 805 Northbound Freeway entrance.'],
      dtype=object)>

In [73]:
model = keras.models.load_model('../data/tf_model.pkl')

In [74]:
y_pred = model.predict(test_data)

1/1 [==============================] - 0s 127ms/step


In [75]:
results_df = pd.DataFrame(y_pred)
results_df.columns = ['72 Hour Violation',
 'COVID-19',
 'Container Left Out',
 'Container Out Late',
 'Damaged/Knocked Over Pole',
 'Dead Animal',
 'Development Services - Code Enforcement',
 'Drain Inlet',
 'Encampment',
 'Encroachment',
 'Environmental Services Code Compliance',
 'Fallen/Hanging Tree Limb',
 'Flashing Traffic Signal Lights',
 'Graffiti Removal',
 'Graffiti Removal - Commercial',
 'Illegal Dumping',
 'Litter',
 'Missed Collection',
 'Other',
 'Oversized Vehicle',
 'Parking',
 'Parking Zone Violation',
 'Pavement Maintenance',
 'Potential Missed Collection',
 'Pothole',
 'Quality of Life Issues',
 'ROW Maintenance',
 'Resurfacing Evaluation',
 'Right-of-Way Code Enforcement',
 'Shared Mobility Device',
 'Sidewalk Repair Issue',
 'Stormwater',
 'Stormwater Code Enforcement',
 'Stormwater Pollution Prevention',
 'Street Flooded',
 'Street Light Maintenance',
 'Street Light Out of Service',
 'Street Sweeping',
 'Traffic Engineering',
 'Traffic Sign Maintenance',
 'Traffic Signal Issue',
 'Traffic Signal Out of Service',
 'Traffic Signal Timing',
 'Trash/Recycling Collection',
 'Tree Maintenance',
 'Tree Removal',
 'Tree Trimming for Pedestrian/Vehicle Clearance',
 'Trimming Request',
 'Vegetation Encroachment',
 'Waste on Private Property',
 'Weed Cleanup']
results_df

,72 Hour Violation,COVID-19,Container Left Out,Container Out Late,Damaged/Knocked Over Pole,Dead Animal,Development Services - Code Enforcement,Drain Inlet,Encampment,Encroachment,...,Traffic Signal Out of Service,Traffic Signal Timing,Trash/Recycling Collection,Tree Maintenance,Tree Removal,Tree Trimming for Pedestrian/Vehicle Clearance,Trimming Request,Vegetation Encroachment,Waste on Private Property,Weed Cleanup
0,3.467675e-08,2.619601e-08,1.839487e-09,2.328662e-21,1.209192e-20,3.033050e-09,7.539589e-07,9.253165e-12,0.99705,0.000004,...,6.557356e-21,1.577459e-25,1.523771e-08,1.837790e-16,1.380510e-18,2.938927e-12,2.340972e-20,2.205864e-10,0.001162,5.257022e-12


In [80]:
#return column where value is that above
# j = 0
max_col = []

# for j in range(1):

i=0


while i < results_df.shape[1]:
    if results_df.iloc[j,i] == results_df.iloc[j,:].max():
        max_col.append(list(results_df.columns)[i])
    i += 1

In [81]:
max_col

['Encampment']